In [18]:
# magics: ensures that any changes to the modules loaded below will be re-loaded automatically
%load_ext autoreload
%autoreload 2

# load general packages
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('seaborn-whitegrid')
import pandas as pd
import tools

# load modules related to this exercise
import model_project as model

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
# solve the model
par = model.setup()
par.theta0 = 0
par.theta1 = 0.2

par = model.create_grids(par)
sol = model.solve(par)

/usr/local/Caskroom/miniconda/base/lib/python3.9/site-packages/scipy/optimize/optimize.py:282: RuntimeWarning: Values in x were outside bounds during a minimize step, clipping to bounds
  warnings.warn("Values in x were outside bounds during a "


In [ ]:
# Define a function for the solution
def plot_solution(sol,par,t):
    fig = plt.figure(figsize=(10,4))# figsize is in inches...
    
    ax = fig.add_subplot(1,2,1)
    ax.scatter(par.grid_M,sol.V[t])
    ax.set_xlabel(f"$M_{t+1}$")
    ax.set_ylabel(f"$V_{t+1}$")

    ax_right = fig.add_subplot(1,2,2)
    ax_right.plot(par.grid_M,sol.c1[t])
    ax_right.plot(par.grid_M,sol.c2[t])
    ax_right.plot(par.grid_M,par.grid_M,color='black',linestyle='dashed')
    ax_right.set_xlabel(f"$M_{t+1}$")
    ax_right.set_ylabel(f"$C_{t+1}$")
    ax_right.legend(['$C^1$','$C^2$'])
    plt.show()

plot_solution(sol,par,0)